### Imports and Dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import io
import requests
import os

from scipy.stats import zscore
from sklearn import preprocessing, metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

Panda Preferences

In [2]:
# Set options for displaying maximum columns and rows
pd.set_option('display.max_columns', None)  
#pd.set_option('display.max_rows', None)

**Read CSV**

In [3]:
# Read the CSV file into a DataFrame
data = pd.read_csv("../Datasets/Farm-Flows.csv")

# Display the DataFrame
display(data)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,fwd_header_size_max,bwd_header_size_tot,bwd_header_size_min,bwd_header_size_max,flow_FIN_flag_count,flow_SYN_flag_count,flow_RST_flag_count,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_URG_flag_count,bwd_URG_flag_count,flow_CWR_flag_count,flow_ECE_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.min,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,traffic,is_attack
0,192.168.1.13,37985,192.168.1.5,1883,tcp,mqtt,0.203465,151,32,OTH,T,T,0,DadA,10,671,10,552,-,0.014749,5,5,5,5,339.004882,339.004882,678.009764,1.000000,40,8,8,40,8,8,0,0,0,0,0,0,0,0,0,0,34.0,40.0,180.0,36.000000,2.345208,22.0,38.0,164.0,32.800000,7.429670,22.0,40.0,344.0,34.400000,5.460973,77.962875,1.511574e+02,4.560947e+02,114.023685,40.448036,2735.137939,3.143072e+03,1.155210e+04,2888.023853,177.765781,77.962875,3.143072e+03,1.474905e+04,1638.783349,1451.562027,23323.535870,5.000000,5.0,180.000000,164.0,180.0,164.0,5.0,5.0,394654.845792,14196.558644,1.474905e+04,1.474905e+04,1.474905e+04,1.474905e+04,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,Normal,0
1,192.168.1.4,51235,192.168.1.5,1880,tcp,http,0.013874,181,0,S1,T,T,0,ShADa,4,345,3,124,-,4.231047,15,15,10,5,3.545222,3.545222,7.090443,1.000000,300,20,20,300,20,20,0,0,0,10,5,30,0,0,0,0,0.0,34.0,250.0,16.666667,14.375906,0.0,30.0,150.0,10.000000,14.638501,0.0,34.0,400.0,13.333333,14.653076,6202.936172,1.072517e+06,4.190391e+06,299313.647406,441206.514988,2056.837082,1.092385e+06,4.221134e+06,301509.567669,473817.757851,40.054321,1.072517e+06,4.231047e+06,145898.177706,333811.857889,94.539244,7.500000,7.5,125.000000,75.0,0.0,0.0,0.0,0.0,0.000000,0.000000,4.231047e+06,4.231047e+06,4.231047e+06,4.231047e+06,0.0,0.0,0.0,0.0,0.0,0.0,2081,64048,1961,64048,Normal,0
2,192.168.1.4,63023,192.168.1.5,1880,tcp,-,0.000520,0,0,SF,T,T,0,FfA,2,80,1,40,-,4.388822,15,14,10,5,3.417774,3.189922,6.607696,0.933333,300,20,20,280,20,20,0,0,0,10,5,29,0,0,0,0,0.0,34.0,250.0,16.666667,14.375906,0.0,30.0,150.0,10.714286,14.917355,0.0,34.0,400.0,13.793103,14.690569,5327.939987,1.009849e+06,4.388822e+06,313487.291336,390104.262534,1737.833023,1.028327e+06,4.173141e+06,321010.846358,472242.170716,27.894974,9.713302e+05,4.388822e+06,156743.645668,308099.533449,91.140628,15.000000,14.0,250.000000,150.0,0.0,0.0,0.0,0.0,0.000000,0.000000,4.388822e+06,4.388822e+06,4.388822e+06,4.388822e+06,0.0,0.0,0.0,0.0,0.0,0.0,2081,64856,1931,64856,Normal,0
3,192.168.1.9,57223,192.168.1.5,1883,tcp,mqtt,4.107098,250,150,OTH,T,T,0,DadA,14,810,15,750,-,0.016622,4,3,2,0,240.647417,180.485563,421.132980,0.750000,84,20,24,64,20,24,0,2,0,2,0,6,0,0,0,0,0.0,175.0,181.0,45.250000,86.546230,0.0,0.0,0.0,0.000000,0.000000,0.0,175.0,181.0,25.857143,65.803821,81.062317,1.534891e+04,1.660085e+04,5533.615748,8517.742779,38.862228,1.647210e+04,1.651096e+04,8255.481720,11620.054722,17.881393,1.523805e+04,1.662183e+04,2770.30468

Fix Dataframe Mixed Types

In [4]:
data.groupby('is_attack')['is_attack'].count()

is_attack
0      27458
1    1282429
Name: is_attack, dtype: int64

In [5]:
data.groupby('traffic')['traffic'].count()

traffic
Arp_Spoofing       2967
BotNet_DDOS          64
HTTP_Flood       265025
ICMP_Flood        76849
MQTT_Flood       132506
Normal            27458
Port_Scanning    126700
TCP_Flood        381765
UDP_Flood        296553
Name: traffic, dtype: int64

In [6]:
data = data.drop('duration', axis=1)

In [7]:
data.groupby('is_attack')['is_attack'].count()

is_attack
0      27458
1    1282429
Name: is_attack, dtype: int64

In [8]:
data.groupby('traffic')['traffic'].count()

traffic
Arp_Spoofing       2967
BotNet_DDOS          64
HTTP_Flood       265025
ICMP_Flood        76849
MQTT_Flood       132506
Normal            27458
Port_Scanning    126700
TCP_Flood        381765
UDP_Flood        296553
Name: traffic, dtype: int64

In [9]:
data = data[~data['traffic'].isin(['BotNet_DDOS'])]

In [10]:
data.groupby('is_attack')['is_attack'].count()

is_attack
0      27458
1    1282365
Name: is_attack, dtype: int64

In [11]:
data.groupby('traffic')['traffic'].count()

traffic
Arp_Spoofing       2967
HTTP_Flood       265025
ICMP_Flood        76849
MQTT_Flood       132506
Normal            27458
Port_Scanning    126700
TCP_Flood        381765
UDP_Flood        296553
Name: traffic, dtype: int64

-----------------------------------------------------------

## DF Statistics and Info

In [12]:
def display_information_dataframe(df_cop):
    # Create a summary of data types, column names, and unique values
    summary_data = [{'Data Type': dtype, 'Column Name': col, 'Unique Values': df_cop[col].unique()} for col, dtype in df_cop.dtypes.iteritems()]
    
    # Create a DataFrame from the summary data
    summary_df = pd.DataFrame(summary_data)
    
    # Set display options to show all rows and columns
    pd.options.display.max_rows = None
    pd.options.display.max_columns = None
    
    # Return the summary DataFrame
    return summary_df

In [13]:
display_information_dataframe(data)

,Data Type,Column Name,Unique Values
0,object,id.orig_h,"[192.168.1.13, 192.168.1.4, 192.168.1.9, 192.1..."
1,int64,id.orig_p,"[37985, 51235, 63023, 57223, 50213, 51319, 604..."
2,object,id.resp_h,[192.168.1.5]
3,int64,id.resp_p,"[1883, 1880, 6901, 31337, 65389, 6123, 16113, ..."
4,object,proto,"[tcp, udp, icmp]"
5,object,service,"[mqtt, http, -, radius, dns, dtls, syslog, dhc..."
6,object,orig_bytes,"[151, 181, 0, 250, 200, 207, 182, 353, 171, 26..."
7,object,resp_bytes,"[32, 0, 150, 120, 37, 294, 59, 64, -, 180, 240..."
8,object,conn_state,"[OTH, S1, SF, RSTRH, SHR, RSTO, S0, SH, RSTOS0..."
9,object,local_orig,[T]


--------------------------------------------

## Pre-processing

Usage Functions:

In [14]:
def remove_outliers(df,columns,n_std):
    for col in columns:
        print(f'[REMOVED OUTLIERS] {col}')
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

In [15]:
def one_hot_encoding(df, columns):
    for col in columns:
        print(f'[ONE HOT ENCODING] {col}')
        df = pd.get_dummies(df, columns=[col], prefix=col)  # Perform one-hot encoding on the column
    
    return df

In [16]:
def zscore_normalization(df, cols):
    # Standardize the selected columns
    for col in cols:
        if col not in df.columns:
            print(f"[WARNING] {col} not found in DataFrame.")
            continue
        print(f"[Z-SCORE] {col}")
        df[col] = zscore(df[col])
    
    print("[DONE] Z-score Normalization")
    print("[INFO] Current Fields in the DataFrame:")
    return df

In [17]:
def delete_columns(df, cols):
    for col in cols:
        df.drop(col, axis = 1, inplace = True)
        print(f'[REMOVED] {col}')
    
    return df

missed_bytes

In [18]:
scaler = MinMaxScaler()
data[['missed_bytes']] = scaler.fit_transform(data[['missed_bytes']])

orig_pkts, resp_pkts

In [19]:
columns = ['orig_pkts', 'resp_pkts']

data = remove_outliers(data, columns, 3)
data = zscore_normalization(data, columns)

[REMOVED OUTLIERS] orig_pkts
[REMOVED OUTLIERS] resp_pkts
[Z-SCORE] orig_pkts
[Z-SCORE] resp_pkts
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


orig_ip_bytes, resp_ip_bytes

In [20]:
columns = ['orig_ip_bytes', 'resp_ip_bytes']

data = remove_outliers(data, columns, 3)
data = zscore_normalization(data, columns)

[REMOVED OUTLIERS] orig_ip_bytes
[REMOVED OUTLIERS] resp_ip_bytes
[Z-SCORE] orig_ip_bytes
[Z-SCORE] resp_ip_bytes
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


fwd_pkts_tot, bwd_pkts_tot

In [21]:
columns = ['fwd_pkts_tot', 'bwd_pkts_tot']
new_columns = ['pkts_difference','pkts_ratio']

data['pkts_difference'] = data['fwd_pkts_tot'] - data['bwd_pkts_tot']

data = zscore_normalization(data, new_columns)
data = delete_columns(data,columns)

[Z-SCORE] pkts_difference
[WARNING] pkts_ratio not found in DataFrame.
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:
[REMOVED] fwd_pkts_tot
[REMOVED] bwd_pkts_tot


fwd_data_pkts_tot, bwd_data_pkts_tot

In [22]:
columns = ['fwd_data_pkts_tot', 'bwd_data_pkts_tot']
new_columns = ['data_pkts_difference','data_pkts_ratio']

data['data_pkts_difference'] = data['fwd_data_pkts_tot'] - data['bwd_data_pkts_tot']

data = zscore_normalization(data, new_columns)
data = delete_columns(data,columns)

[Z-SCORE] data_pkts_difference
[WARNING] data_pkts_ratio not found in DataFrame.
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:
[REMOVED] fwd_data_pkts_tot
[REMOVED] bwd_data_pkts_tot


fwd_pkts_per_sec, bwd_pkts_per_sec, flow_pkts_per_sec

In [23]:
columns = ['fwd_pkts_per_sec', 'bwd_pkts_per_sec', 'flow_pkts_per_sec']

data = remove_outliers(data, columns, 3)
data[columns] = scaler.fit_transform(data[columns])

[REMOVED OUTLIERS] fwd_pkts_per_sec
[REMOVED OUTLIERS] bwd_pkts_per_sec
[REMOVED OUTLIERS] flow_pkts_per_sec


down_up_ratio

In [24]:
columns = ['down_up_ratio']
data[columns] = scaler.fit_transform(data[columns])

fwd_header_size_min, fwd_header_size_max

In [25]:
columns = ['fwd_header_size_min', 'fwd_header_size_max','fwd_header_size_tot']

data = zscore_normalization(data, columns)

[Z-SCORE] fwd_header_size_min
[Z-SCORE] fwd_header_size_max
[Z-SCORE] fwd_header_size_tot
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


bwd_header_size_min, bwd_header_size_max

In [26]:
columns = ['bwd_header_size_min', 'bwd_header_size_max','bwd_header_size_tot']

data = zscore_normalization(data, columns)

[Z-SCORE] bwd_header_size_min
[Z-SCORE] bwd_header_size_max
[Z-SCORE] bwd_header_size_tot
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


fwd_pkts_payload.tot, fwd_pkts_payload.avg

In [27]:
columns = ['fwd_pkts_payload.tot', 'fwd_pkts_payload.avg']

data = zscore_normalization(data, columns)

[Z-SCORE] fwd_pkts_payload.tot
[Z-SCORE] fwd_pkts_payload.avg
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


bwd_pkts_payload.tot,bwd_pkts_payload.avg

In [28]:
columns = ['bwd_pkts_payload.tot','bwd_pkts_payload.avg']

data = zscore_normalization(data, columns)

[Z-SCORE] bwd_pkts_payload.tot
[Z-SCORE] bwd_pkts_payload.avg
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


flow_pkts_payload.tot,flow_pkts_payload.avg

In [29]:
columns = ['flow_pkts_payload.tot','flow_pkts_payload.avg']

data = zscore_normalization(data, columns)

[Z-SCORE] flow_pkts_payload.tot
[Z-SCORE] flow_pkts_payload.avg
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


fwd_iat.tot,fwd_iat.avg

In [30]:
columns = ['fwd_iat.tot','fwd_iat.avg']

data = zscore_normalization(data, columns)

[Z-SCORE] fwd_iat.tot
[Z-SCORE] fwd_iat.avg
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


bwd_iat.tot,bwd_iat.avg

In [31]:
columns = ['bwd_iat.tot','bwd_iat.avg']

data = zscore_normalization(data, columns)

[Z-SCORE] bwd_iat.tot
[Z-SCORE] bwd_iat.avg
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


flow_iat.tot,flow_iat.avg

In [32]:
columns = ['flow_iat.tot','flow_iat.avg']

data = zscore_normalization(data, columns)

[Z-SCORE] flow_iat.tot
[Z-SCORE] flow_iat.avg
[DONE] Z-score Normalization
[INFO] Current Fields in the DataFrame:


Unused Columns

In [33]:
columns = [
    'id.orig_h',
    'id.orig_p',
    'id.resp_h',
    'id.resp_p',
    'proto',
    'service',
    'orig_bytes',
    'resp_bytes',
    'conn_state',
    'local_orig',
    'local_resp',
    'history',
    'tunnel_parents',
    'flow_duration',
    'flow_FIN_flag_count',
    'flow_SYN_flag_count',
    'flow_RST_flag_count',
    'fwd_PSH_flag_count',
    'bwd_PSH_flag_count',
    'flow_ACK_flag_count',
    'fwd_URG_flag_count',
    'bwd_URG_flag_count',
    'flow_CWR_flag_count',
    'flow_ECE_flag_count',
    'fwd_pkts_payload.min',
    'fwd_pkts_payload.max',
    'fwd_pkts_payload.std',
    'bwd_pkts_payload.min',
    'bwd_pkts_payload.max',
    'bwd_pkts_payload.std',
    'flow_pkts_payload.min',
    'flow_pkts_payload.max',
    'flow_pkts_payload.std',
    'fwd_iat.min',
    'fwd_iat.max',
    'fwd_iat.std',
    'bwd_iat.min',
    'bwd_iat.max',
    'bwd_iat.std',
    'flow_iat.min',
    'flow_iat.max',
    'flow_iat.std',
    'payload_bytes_per_second',
    'fwd_subflow_pkts',
    'bwd_subflow_pkts',
    'fwd_subflow_bytes',
    'bwd_subflow_bytes',
    'fwd_bulk_bytes',
    'bwd_bulk_bytes',
    'fwd_bulk_packets',
    'bwd_bulk_packets',
    'fwd_bulk_rate',
    'bwd_bulk_rate',
    'active.min',
    'active.max',
    'active.tot',
    'active.avg',
    'active.std',
    'idle.min',
    'idle.max',
    'idle.tot',
    'idle.avg',
    'idle.std',
    'fwd_init_window_size',
    'bwd_init_window_size',
    'fwd_last_window_size',
    'bwd_last_window_size',
]

data = delete_columns(data,columns)

[REMOVED] id.orig_h
[REMOVED] id.orig_p
[REMOVED] id.resp_h
[REMOVED] id.resp_p
[REMOVED] proto
[REMOVED] service
[REMOVED] orig_bytes
[REMOVED] resp_bytes
[REMOVED] conn_state
[REMOVED] local_orig
[REMOVED] local_resp
[REMOVED] history
[REMOVED] tunnel_parents
[REMOVED] flow_duration
[REMOVED] flow_FIN_flag_count
[REMOVED] flow_SYN_flag_count
[REMOVED] flow_RST_flag_count
[REMOVED] fwd_PSH_flag_count
[REMOVED] bwd_PSH_flag_count
[REMOVED] flow_ACK_flag_count
[REMOVED] fwd_URG_flag_count
[REMOVED] bwd_URG_flag_count
[REMOVED] flow_CWR_flag_count
[REMOVED] flow_ECE_flag_count
[REMOVED] fwd_pkts_payload.min
[REMOVED] fwd_pkts_payload.max
[REMOVED] fwd_pkts_payload.std
[REMOVED] bwd_pkts_payload.min
[REMOVED] bwd_pkts_payload.max
[REMOVED] bwd_pkts_payload.std
[REMOVED] flow_pkts_payload.min
[REMOVED] flow_pkts_payload.max
[REMOVED] flow_pkts_payload.std
[REMOVED] fwd_iat.min
[REMOVED] fwd_iat.max
[REMOVED] fwd_iat.std
[REMOVED] bwd_iat.min
[REMOVED] bwd_iat.max
[REMOVED] bwd_iat.std
[REM

In [34]:
display_information_dataframe(data)

,Data Type,Column Name,Unique Values
0,float64,missed_bytes,"[0.0, 0.2294239391026379, 0.43031003267600554,..."
1,float64,orig_pkts,"[1.3527436362558154, 0.40572502783592496, 1.82..."
2,float64,orig_ip_bytes,"[2.6965191769759236, 0.11324071692116469, 2.70..."
3,float64,resp_pkts,"[3.6472734143478704, 0.8822289033247395, -0.50..."
4,float64,resp_ip_bytes,"[4.328449638419265, 1.0326147694349792, 4.1715..."
5,float64,fwd_pkts_per_sec,"[7.776270484850156e-05, 7.49671955101493e-05, ..."
6,float64,bwd_pkts_per_sec,"[0.00010565584897994996, 9.506708383560181e-05..."
7,float64,flow_pkts_per_sec,"[0.00010565583407878875, 9.84623432159424e-05,..."
8,float64,down_up_ratio,"[0.16666666666666666, 0.15555549999999999, 0.0..."
9,float64,fwd_header_size_tot,"[4.533276933712789, 0.2269995085143721, 4.8645..."


---------------------------------------

## Split Dataset

In [35]:
X_columns = data.columns.drop('traffic')

le = LabelEncoder()
le.fit(data["traffic"].values)

X = data[X_columns].values
y = data["traffic"].values
y = le.transform(y)

In [36]:
# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [37]:
x_test.shape,y_test.shape

((279855, 30), (279855,))

In [38]:
x_train.shape,y_train.shape

((839563, 30), (839563,))

In [39]:
# Convert arrays to Series or DataFrame
x_train = pd.DataFrame(x_train, columns=X_columns)
y_train = pd.Series(y_train, name='traffic')
x_test = pd.DataFrame(x_test, columns=X_columns)
y_test = pd.Series(y_test, name='traffic')

In [40]:
# Create new DataFrames for training and testing sets
train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

In [41]:
train_df.groupby('traffic')['traffic'].count()

traffic
0      2019
1     84721
2     57611
3     92991
4      5321
5     94930
6    279559
7    222411
Name: traffic, dtype: int64

In [42]:
test_df.groupby('traffic')['traffic'].count()

traffic
0      673
1    28240
2    19204
3    30997
4     1774
5    31643
6    93187
7    74137
Name: traffic, dtype: int64

---------------------------------------

## Oversample and Undersampling

In [43]:
def create_balanced_dataset(data, target_class, sample_size, random_state=42):
    filtered_data = data
    
    if target_class > -1:    
        # Step 1: Filter out rows with the target class
        filtered_data = data[data['traffic'] != target_class]
    
    # Step 2: Randomly sample around 'sample_size' values from each class (except the target class)
    def sample_or_all(group, n=sample_size):
        return group.sample(n=n, random_state=random_state) if len(group) >= n else group

    balanced_dataset = filtered_data.groupby('traffic').apply(sample_or_all)
    
    # Step 3: Reset the index to create the balanced dataset without any unwanted index columns
    balanced_dataset = balanced_dataset.reset_index(drop=True)
    
    # Step 4: Get the rows with the target class
    target_class_data = data[data['traffic'] == target_class]
    
    # Step 5: Concatenate the balanced dataset with the rows from the target class
    final_dataset = pd.concat([balanced_dataset, target_class_data])
    
    return final_dataset

### Training Set

In [44]:
train_df.groupby('is_attack')['is_attack'].count()

is_attack
0.0      5321
1.0    834242
Name: is_attack, dtype: int64

In [45]:
train_df.groupby('traffic')['traffic'].count()

traffic
0      2019
1     84721
2     57611
3     92991
4      5321
5     94930
6    279559
7    222411
Name: traffic, dtype: int64

In [46]:
# Calculate the maximum count among all classes
max_count = train_df['traffic'].value_counts().max()

# Filter the rows corresponding to the 'Normal' class
normal_class_data = train_df[train_df['traffic'] == 4]

# Determine the number of times to duplicate the 'Normal' class rows
num_duplicates = max_count // len(normal_class_data) - 1

# Duplicate the 'Normal' class rows
oversampled_normal_class_data = pd.concat([normal_class_data] * num_duplicates, ignore_index=True)

# Calculate how many additional rows are needed to reach the maximum count
remaining_rows_needed = max_count - len(oversampled_normal_class_data)

# Randomly sample the required number of additional rows from the 'Normal' class
additional_normal_rows = normal_class_data.sample(n=remaining_rows_needed, replace=True)

# Combine the oversampled 'Normal' class data with the original data
oversampled_data = pd.concat([train_df, oversampled_normal_class_data, additional_normal_rows], ignore_index=True)

In [47]:
oversampled_data.groupby('traffic')['traffic'].count()

traffic
0      2019
1     84721
2     57611
3     92991
4    284880
5     94930
6    279559
7    222411
Name: traffic, dtype: int64

In [48]:
oversampled_data.groupby('is_attack')['is_attack'].count()

is_attack
0.0    284880
1.0    834242
Name: is_attack, dtype: int64

In [49]:
training_set_binary = create_balanced_dataset(oversampled_data, target_class=4, sample_size=45697)

In [50]:
training_set_multiclass = create_balanced_dataset(oversampled_data, target_class=0, sample_size=57611)

In [51]:
training_set_binary.groupby('is_attack')['is_attack'].count()

is_attack
0.0    284880
1.0    276201
Name: is_attack, dtype: int64

In [52]:
training_set_multiclass.groupby('traffic')['traffic'].count()

traffic
0     2019
1    57611
2    57611
3    57611
4    57611
5    57611
6    57611
7    57611
Name: traffic, dtype: int64

### Testing Set

In [53]:
test_df.groupby('is_attack')['is_attack'].count()

is_attack
0.0      1774
1.0    278081
Name: is_attack, dtype: int64

In [54]:
test_df.groupby('traffic')['traffic'].count()

traffic
0      673
1    28240
2    19204
3    30997
4     1774
5    31643
6    93187
7    74137
Name: traffic, dtype: int64

In [55]:
testing_set_binary = create_balanced_dataset(test_df, target_class=4, sample_size=253)

In [56]:
testing_set_multiclass = create_balanced_dataset(test_df, target_class=-1, sample_size=1774)

In [57]:
testing_set_binary.groupby('is_attack')['is_attack'].count()

is_attack
0.0    1774
1.0    1771
Name: is_attack, dtype: int64

In [58]:
testing_set_multiclass.groupby('traffic')['traffic'].count()

traffic
0     673
1    1774
2    1774
3    1774
4    1774
5    1774
6    1774
7    1774
Name: traffic, dtype: int64

--------------

## Save Train and Test Dataset

In [59]:
# Directory to save the files
save_dir = '../Datasets/'

# Save train and test DataFrames to CSV files in the specified directory
train_path_bin = os.path.join(save_dir, 'Farm-Flow_Train_Binary.csv')
test_path_bin = os.path.join(save_dir, 'Farm-Flow_Test_Binary.csv')
train_path_mul = os.path.join(save_dir, 'Farm-Flow_Train_Multiclass.csv')
test_path_mul = os.path.join(save_dir, 'Farm-Flow_Test_Multiclass.csv')

training_set_binary.to_csv(train_path_bin, index=False)
testing_set_binary.to_csv(test_path_bin, index=False)

training_set_multiclass.to_csv(train_path_mul, index=False)
testing_set_multiclass.to_csv(test_path_mul, index=False)